In [4]:
import sys
import os
from pathlib import Path
import json

# Obtener la ruta absoluta a la carpeta raíz del proyecto
ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))


# Añadir esa ruta a sys.path
if ROOT not in sys.path:
    sys.path.append(ROOT)

print(ROOT)

# print(os.path.isfile(PROMPTS_DB_PATH))

c:\Formacion\Customer_bot


In [9]:
from bot.config import FAQ_CSV_PATH
from bot.models import BotState, IntentResult, KnowledgeHit, Entity, AnswerResult, NLPResult
from bot.nlp_agent import nlp_node
from bot.graph import initial_state


FAQ_CSV_PATH = os.path.join(ROOT, FAQ_CSV_PATH)

# Añadir esa ruta a sys.path
if FAQ_CSV_PATH not in sys.path:
    sys.path.append(FAQ_CSV_PATH)

print(FAQ_CSV_PATH)

c:\Formacion\Customer_bot\data/faq.csv


In [63]:
intent = IntentResult(
    tipo_mensaje="pregunta",
    intencion="canarias",
    confianza=0.92,
    sentimiento="neutral"
)

ent1 = Entity(tipo="producto", valor="auriculares")
ent2 = Entity(tipo="numero_pedido", valor="XYZ123")
ent3 = Entity(tipo="ciudad", valor="Madrid")

nlp_result = NLPResult(
    intent=intent,
    entidades=[ent1, ent2, ent3]
)

hit = KnowledgeHit(
    categoria="envíos",
    pregunta_canonica="¿Dónde está mi pedido?",
    respuesta_base="Tu pedido está en tránsito.",
    score=0.87
)

answer = AnswerResult(
    respuesta="Tu pedido llegará mañana.",
    necesita_revision_humano=False,
    razon=None,
    metadata={"fuente": "FAQ base"}
)

Bot_state =  BotState(
    user_message="¿Dónde está mi pedido?",
    nlp = nlp_result,
    knowledge_hits=[],
    answer=answer,
    debug={}
)

In [64]:

import csv
from typing import List, Dict, Any
from bot.knowledge import simple_match_score

#load_faq()
faq_rows: List[Dict[str, Any]] = []
with open(FAQ_CSV_PATH, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        faq_rows.append(row)

print(f"\nTotal filas cargadas: {len(faq_rows)}\n")
for r in faq_rows[:5]:
    print(r)



Total filas cargadas: 3

{'categoria': 'envios', 'pregunta_canonica': '¿Cuánto tardan los envíos a España?', 'respuesta_base': 'Los envíos a España peninsular tardan entre 24 y 48 horas laborables.'}
{'categoria': 'envios_canarias', 'pregunta_canonica': '¿Envían a Canarias?', 'respuesta_base': 'Sí', None: [' enviamos a Canarias. El plazo de entrega es de 5 a 7 días laborables y puede haber sobrecoste de aduanas.']}
{'categoria': 'devoluciones', 'pregunta_canonica': '¿Cuál es vuestra política de devoluciones?', 'respuesta_base': 'Aceptamos devoluciones durante los primeros 30 días posteriores a la recepción del pedido.'}


In [65]:
intencion  = Bot_state.nlp.intent.intencion
print(intencion)
hits: List[Dict[str, Any]] = []
for row in faq_rows:
        categoria = row.get("categoria", "")
        score = simple_match_score(intencion, categoria)
       
        if score > 0:
            hits.append(
                {
                    "categoria": categoria,
                    "pregunta_canonica": row.get("pregunta_canonica", ""),
                    "respuesta_base": row.get("respuesta_base", ""),
                    "score": score,
                }
            )
        print(categoria)
        print(score) 
        print(hits)    
     

canarias
envios
0.0
[]
envios_canarias
1.0
[{'categoria': 'envios_canarias', 'pregunta_canonica': '¿Envían a Canarias?', 'respuesta_base': 'Sí', 'score': 1.0}]
devoluciones
0.0
[{'categoria': 'envios_canarias', 'pregunta_canonica': '¿Envían a Canarias?', 'respuesta_base': 'Sí', 'score': 1.0}]


In [66]:
hits.sort(key=lambda h: h["score"], reverse=True)
top_hits = hits[:3]
print(f"\nTotal hits encontrados:\n{len(hits)}.\nTop 3: {top_hits}")


Total hits encontrados:
1.
Top 3: [{'categoria': 'envios_canarias', 'pregunta_canonica': '¿Envían a Canarias?', 'respuesta_base': 'Sí', 'score': 1.0}]
